In [ ]:
# 1. Instalación de librerías necesarias
# Descomenta la siguiente línea si no tienes instaladas estas librerías
# !pip install transformers datasets torch scikit-learn pandas accelerate

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

# Verificar si hay GPU disponible (recomendado para entrenar BERT)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

In [ ]:
# 2. Carga de Datos
# Cargamos el dataset procesado
file_path = './1. Detector_cyberbullying/cyberbullying_tweets_processed.csv'
df = pd.read_csv(file_path)

# Usaremos 'tweet_text_clean' como entrada y 'cyberbullying_type' como etiqueta
# Eliminamos filas con nulos en el texto limpio y en la etiqueta por seguridad
df = df.dropna(subset=['tweet_text_clean', 'cyberbullying_type'])

# Codificar las etiquetas de texto a números
le = LabelEncoder()
df['label'] = le.fit_transform(df['cyberbullying_type'])

# Guardar el mapeo de etiquetas para luego
# Usamos range(len(le.classes_)) para evitar errores de transformación redundante
label_map = dict(zip(le.classes_, range(len(le.classes_))))
print("Mapeo de etiquetas:", label_map)

# Separar textos y etiquetas
texts = df['tweet_text_clean'].tolist()
labels = df['label'].tolist()

# División Train/Test (80% train, 20% test)
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

print(f"Datos de entrenamiento: {len(train_texts)}")
print(f"Datos de prueba: {len(test_texts)}")

In [ ]:
# 3. Tokenización
# Cargamos el tokenizador pre-entrenado de DistilBERT
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenizamos los textos
# truncation=True: Corta los tweets que sean más largos que el máximo permitido
# padding=True: Rellena los tweets cortos para que todos tengan la misma longitud
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [ ]:
# Crear una clase Dataset compatible con PyTorch
class CyberbullyingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CyberbullyingDataset(train_encodings, train_labels)
test_dataset = CyberbullyingDataset(test_encodings, test_labels)

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Configuración del flujo de trabajo
utilizar_pesos_guardados = True  # True: Intenta cargar modelo guardado. False: Fuerza re-entrenamiento.
model_path = "./modelo_ciberbullying_distilbert"

In [ ]:
# 4. Lógica de Modelo (Cargar o Entrenar)
import os
import torch
from sklearn.metrics import accuracy_score
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Definir dispositivo aquí también por seguridad
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Asegurar variables por si no se ejecutó la celda anterior
if 'utilizar_pesos_guardados' not in locals():
    utilizar_pesos_guardados = False
if 'model_path' not in locals():
    model_path = "./modelo_ciberbullying_distilbert"

num_labels = len(label_map)
model_loaded = False

# Comprobar si existe el modelo guardado
if utilizar_pesos_guardados and os.path.exists(model_path):
    print(f"Cargando modelo guardado desde '{model_path}'...")
    try:
        model = DistilBertForSequenceClassification.from_pretrained(model_path)
        # model.to(device) # Eliminado: Trainer gestiona el dispositivo automáticamente
        model_loaded = True
        print("Modelo cargado exitosamente.")
    except Exception as e:
        print(f"Error cargando el modelo: {e}. Se procederá a entrenar uno nuevo.")
        model_loaded = False

if not model_loaded:
    print("Iniciando configuración para entrenamiento de nuevo modelo...")
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
    # model.to(device) # Eliminado: Trainer gestiona el dispositivo automáticamente
    
    # Configuración del Entrenamiento
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=2,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        acc = accuracy_score(labels, preds)
        return {'accuracy': acc}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )
    
    print("Entrenando modelo...")
    trainer.train()
    
    print(f"Guardando modelo en '{model_path}'...")
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
else:
    # Si cargamos el modelo, instanciamos un Trainer para evaluación
    training_args = TrainingArguments(
        output_dir='./results',
        per_device_eval_batch_size=64
    )
    
    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        acc = accuracy_score(labels, preds)
        return {'accuracy': acc}

    trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

In [ ]:
# 5. Evaluación y Métricas
import numpy as np
from sklearn.metrics import classification_report

print("Evaluando modelo...")
results = trainer.evaluate()
print("Resultados de evaluación:", results)

# Predicciones sobre el conjunto de test
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=-1)
y_true = test_labels

# Asegurar orden correcto de etiquetas para el reporte
target_names = [k for k, v in sorted(label_map.items(), key=lambda item: item[1])]

print("\nReporte de Clasificación:")
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
# 6. Gráficos y Estadísticas
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Obtener los índices y nombres de las etiquetas en orden
labels_indices = [v for k, v in sorted(label_map.items(), key=lambda item: item[1])]
labels_names = [k for k, v in sorted(label_map.items(), key=lambda item: item[1])]

# Generar matriz con etiquetas explícitas para asegurar el tamaño correcto
# Esto evita errores si alguna clase no aparece en las predicciones o en el test set
cm = confusion_matrix(y_true, y_pred, labels=labels_indices)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels_names, 
            yticklabels=labels_names)
plt.title('Matriz de Confusión - DistilBERT')
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.show()